In [1]:
import signac
import numpy as np
import matplotlib as plt
import freud
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd

In [26]:
data_path = "Projects/ptb7-itic/"
project = signac.get_project(data_path)
for job in project:
    print(job)

fe36ee4052a01cae041ef5988477de10


In [29]:
state_dict = {"process": "quench", "size": "medium"}
job_list = project.find_jobs(state_dict)
job_list

,sp.molecule,sp.n_compounds,sp.density,sp.e_factor,sp.kT_reduced,sp.tau,sp.n_steps,sp.dt,sp.remove_hydrogens,sp.original_id,...,doc.mass,doc.mass_units,doc.energy,doc.energy_units,doc.distance,doc.distance_units,doc.T_SI,doc.T_unit,doc.real_timestep,doc.time_unit
fe36ee4052a01cae041ef5988477de10,PTB7_10mer_smiles,200,0.9,0.5,1.7,1,60000000.0,0.001,True,ecca9fd02b6f816abf804563103b7ef8,...,32.06,amu,1.046,kj/mol,0.356359,nm,214.0,K,1.973,fs


In [43]:
for job in job_list:
    path = (job.fn("trajectory.gsd"))
    print(path)
    rdf_from_gsd(job)

/Users/emilyepstein/Projects/ptb7-itic/workspace/fe36ee4052a01cae041ef5988477de10/trajectory.gsd


RuntimeError: Not a GSD file: <_io.BufferedReader name='/Users/emilyepstein/Projects/ptb7-itic/workspace/fe36ee4052a01cae041ef5988477de10/trajectory.gsd'>

In [42]:
def atom_type_pos(frame, atom_types): 
    positions = []
    for idx, type_id in enumerate(frame.particles.typeid):
        if frame.particles.types[type_id] in atom_types:
            positions.append(frame.particles.position[idx])
    return positions

def rdf_from_gsd(job, atom_type='ss', r_max=5, r_min=0.5, nbins=50, start=100):
    gsd_file = "{}/trajectory.gsd".format(job.workspace())
    
    f = gsd.pygsd.GSDFile(open(gsd_file, "rb")) 
    trajectory = gsd.hoomd.HOOMDTrajectory(f) 
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    
    for frame in trajectory[start:]:
        freud_rdf.compute(system=frame, reset=False)
    x = freud_rdf.bin_centers
    y = freud_rdf.rdf
    
    filename = 'rdf_info.txt'
    
    np.savetxt(filename, np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
    f.close()
    return freud_rdf
